In [1]:
import datetime
import logging
import os
import random
import time
import warnings

import csv
import gluonnlp as nlp
import matplotlib.pyplot as plt
import mxnet as mx
import numpy as np
import pandas as pd
import seaborn as sns

from bert import *
from mxboard import SummaryWriter
from mxnet import gluon
from mxnet.gluon.data import Dataset, SimpleDataset
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score)
from sklearn.model_selection import train_test_split
from sklearn import utils
from tqdm import tqdm

In [2]:
%matplotlib inline

In [3]:
warnings.filterwarnings('ignore')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

In [4]:
# set repeatable random state
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)

In [5]:
# apply progress bars for pandas .apply() -> .progress_apply()
tqdm.pandas()

In [6]:
# make tqdm jupyter friendly
from tqdm import tqdm_notebook as tqdm
# for .progress_apply() we have to hack it like this?
tqdm().pandas()

In [7]:
class Timer:
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.time_start = time.time()

    def __exit__(self, *exc):
        time_end = time.time()
        time_delta = datetime.timedelta(seconds=(time_end - self.time_start))
        if self.name:
            print(("Time for [{}]: {}".format(self.name, time_delta)))
        else:
            print(("Time: {}".format(time_delta)))

---

In [8]:
fn_art_eval = "data/artificial_evalset/artificial_evalset.tsv"

In [9]:
artificial_evalset_df = pd.DataFrame.from_csv(fn_art_eval, sep='\t', index_col=None)

new_cols = artificial_evalset_df.columns.to_list()
new_cols[2] = "type"
artificial_evalset_df.columns = new_cols

In [10]:
def fix_cols(row):
    row["argument1_id"] = row['arg_id']
    row["argument2_id"] = "{}-{}".format(row['arg_id'], row['type'])
    row["topic"] = "gay marriage"
    return row

artificial_evalset_df = artificial_evalset_df.apply(fix_cols, axis=1)

In [11]:
def add_tag(row):
    row["tag"] = "gay marriage"
    return row

artificial_evalset_df = artificial_evalset_df.apply(add_tag, axis=1)

In [12]:
ctx = mx.cpu()
_, vocabulary = nlp.model.get_model('bert_12_768_12',
                                    dataset_name='book_corpus_wiki_en_uncased',
                                    pretrained=True, ctx=ctx, use_pooler=True,
                                    use_decoder=False, use_classifier=False)
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)
tokenizer = bert_tokenizer

In [13]:
# tokenizer from BERT
def tokenize_arguments(row):
    # tokenize
    row['argument1_tokens'] = tokenizer(row['argument1'])
    row['argument2_tokens'] = tokenizer(row['argument2'])

    # count tokens
    row['argument1_len'] = len(row['argument1_tokens'])
    row['argument2_len'] = len(row['argument2_tokens'])
    # token number diff
    row['argument12_len_sum'] = row['argument1_len'] + row['argument2_len']
    row['argument12_len_sum_half'] = row['argument12_len_sum'] / 2
    row['argument12_len_diff'] = row['argument1_len'] - row['argument2_len']
    row['argument12_len_diff_abs'] = np.abs(row['argument12_len_diff'])
    return row


artificial_evalset_df = artificial_evalset_df.progress_apply(tokenize_arguments, axis=1)

In [14]:
from nltk.tokenize import sent_tokenize, word_tokenize
# nltk.download('punct')


# tokenizer from BERT
def sentenize_arguments(row):
    # tokenize
    row['argument1_sentences'] = sent_tokenize(row['argument1'])
    row['argument2_sentences'] = sent_tokenize(row['argument2'])

    # count tokens
    row['argument1_sent_num'] = len(row['argument1_sentences'])
    row['argument2_sent_num'] = len(row['argument2_sentences'])
    # token number diff
    row['argument12_sent_num_sum'] = row['argument1_sent_num'] + row['argument2_sent_num']
    row['argument12_sent_num_sum_half'] = row['argument12_sent_num_sum'] / 2
    row['argument12_sent_num_diff'] = row['argument1_sent_num'] - row['argument2_sent_num']
    row['argument12_sent_num_diff_abs'] = np.abs(row['argument12_sent_num_diff'])
    return row


artificial_evalset_df = artificial_evalset_df.progress_apply(sentenize_arguments, axis=1)

In [15]:
def get_overview(df, task='same-side', class_name='is_same_side'):
    # Total instance numbers
    total = len(df)
    print("Task: ", task)
    print('=' * 40, '\n')

    print('Total instances: ', total)
    print('\n')

    print('For each topic:')
    for tag, tag_df in df.groupby(['tag']):
        print(tag, ': ', len(tag_df), ' instances')
        print('')
        print('\t\tUnique argument1:', len(tag_df['argument1'].unique()))
        print('\t\tUnique argument2:', len(tag_df['argument2'].unique()))
        arguments = np.concatenate([tag_df['argument1'].values, tag_df['argument2'].values])
        print('\t\tUnique total arguments:', len(set(list(arguments))), '\n')
        if class_name in df.columns:
            for is_same_side, side_df in tag_df.groupby([class_name]):
                print('\t\t', is_same_side, ': ', len(side_df), ' instances')
    print('\n')

    if class_name in df.columns:
        print('For each class value:')
        for class_value, class_df in df.groupby([class_name]):
            print(class_value, ': ', len(class_df), ' instances')
            print('\t\tUnique argument1:', len(class_df['argument1'].unique()))
            print('\t\tUnique argument2:', len(class_df['argument2'].unique()))
            arguments = np.concatenate([class_df['argument1'].values, class_df['argument2'].values])
            print('\t\tUnique total arguments:', len(set(list(arguments))), '\n')
        print('\n')

    print('Unique argument1:', len(df['argument1'].unique()))
    print('Unique argument2:', len(df['argument2'].unique()))
    arguments = df['argument1'].values
    arguments = np.concatenate([arguments, df['argument2'].values])

    print('Unique total arguments:', len(set(list(arguments))), '\n')

    print('-' * 40, '\n')

    arguments_length_lst = [x for x in df['argument1_len'].values]
    arguments_length_lst.extend([x for x in df['argument2_len'].values])
    print('Words:')
    print('\tshortest argument:', min(arguments_length_lst), ' words')
    print('\tlongest argument:', max(arguments_length_lst), ' words')
    print('\targument average length:', np.mean(arguments_length_lst),
          ' words')

    arguments_sent_length_lst = [x for x in df['argument1_sent_num'].values]
    arguments_sent_length_lst.extend([x for x in df['argument2_sent_num'].values])
    print('Sentences:')
    print('\tshortest argument:', min(arguments_sent_length_lst), ' sentences')
    print('\tlongest argument:', max(arguments_sent_length_lst), ' sentences')
    print('\targument average length:', np.mean(arguments_sent_length_lst),
          ' sentences')

In [16]:
get_overview(artificial_evalset_df, task="same-side artificial evalset")

Task:  same-side artificial evalset

Total instances:  175


For each topic:
gay marriage :  175  instances

		Unique argument1: 25
		Unique argument2: 174
		Unique total arguments: 199 

		 False :  100  instances
		 True :  75  instances


For each class value:
False :  100  instances
		Unique argument1: 25
		Unique argument2: 100
		Unique total arguments: 125 

True :  75  instances
		Unique argument1: 25
		Unique argument2: 74
		Unique total arguments: 99 



Unique argument1: 25
Unique argument2: 174
Unique total arguments: 199 

---------------------------------------- 

Words:
	shortest argument: 7  words
	longest argument: 137  words
	argument average length: 25.597142857142856  words
Sentences:
	shortest argument: 1  sentences
	longest argument: 5  sentences
	argument average length: 1.3314285714285714  sentences


In [17]:
artificial_evalset_df.describe()

,arg_id,argument1_id,argument1_len,argument2_len,argument12_len_sum,argument12_len_sum_half,argument12_len_diff,argument12_len_diff_abs,argument1_sent_num,argument2_sent_num,argument12_sent_num_sum,argument12_sent_num_sum_half,argument12_sent_num_diff,argument12_sent_num_diff_abs
count,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000
mean,13.000000,13.000000,26.240000,24.954286,51.194286,25.597143,1.285714,8.520000,1.280000,1.382857,2.662857,1.331429,-0.102857,0.182857
std,7.231794,7.231794,26.187423,16.712174,41.693007,20.846504,13.850954,10.977427,0.724291,0.814201,1.479959,0.739980,0.429839,0.402211
min,1.000000,1.000000,7.000000,8.000000,15.000000,7.500000,-26.000000,0.000000,1.000000,1.000000,2.000000,1.000000,-2.000000,0.000000
25%,7.000000,7.000000,15.000000,15.500000,30.000000,15.000000,-6.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000
50%,13.000000,13.000000,19.000000,20.000000,39.000000,19.500000,-1.000000,6.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000
75%,19.000000,19.000000,32.000000,27.000000,54.000000,27.000000,5.000000,10.000000,1.000000,1.000000,3.000000,1.500000,0.000000,0.000000
max,25.000000,25.000000,137.000000,108.000000,245.000000,122.500000,72.000000,72.000000,4.000000,5.000000,9.000000,4.500000,1.000000,2.000000


---

In [18]:
names_columns_X = ['argument1', 'argument2', 'argument1_id', 'argument2_id', 'topic']
names_columns_X2 = ['argument1', 'argument2', 'tag']
names_columns_y = ['is_same_side']

In [19]:
def compute_metrics(conf_mat, precision=3, dump=True):
    conf_mat = np.array(conf_mat)
    tn, fp, fn, tp = conf_mat.ravel()

    acc = (tp + tn) / (tp + tn + fp + fn)
    prec = tp / (tp + fp)
    rec  = tp / (tp + fn)
    f1 = 2 * (prec * rec) / (prec + rec)

    if dump:
        print("{:>10}: {:.{prec}f}".format("accuracy", acc, prec=precision))
        print("{:>10}: {:.{prec}f}".format("precision", prec, prec=precision))
        print("{:>10}: {:.{prec}f}".format("recall", rec, prec=precision))
        print("{:>10}: {:.{prec}f}".format("f1-score", f1, prec=precision))

    return prec, rec, f1, acc

In [20]:
def heatconmat(y_test, y_pred):
    sns.set_context('talk')
    plt.figure(figsize=(9, 6))
    sns.heatmap(confusion_matrix(y_test, y_pred),
                annot=True,
                fmt='d',
                cbar=False,
                cmap='gist_earth_r',
                yticklabels=sorted(np.unique(y_test)))
    plt.show()


def report_training_results(y_test, y_pred, name=None, heatmap=True):
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))
    print()
    compute_metrics(confusion_matrix(y_test, y_pred))
    if heatmap:
        heatconmat(y_test, y_pred)
    print()
    print('Accuracy: ', round(accuracy_score(y_test, y_pred), 2), '\n')  #

    print('Report{}:'.format("" if not name else " for [{}]".format(name)))
    print(classification_report(y_test, y_pred))

    f1_dic = {}
    f1_dic['macro'] = round(
        f1_score(y_pred=y_pred, y_true=y_test, average='macro'), 2)
    f1_dic['micro'] = round(
        f1_score(y_pred=y_pred, y_true=y_test, average='micro'), 2)
    return f1_dic

In [21]:
fn = "data/artificial_evalset/artificial_evalset.pred.tsv"

In [22]:
artificial_evalset_df = pd.DataFrame.from_csv(fn, sep='\t')

In [46]:
# invert NEG:
# abs(v - 1) -- v is value (0|1)


def dump_art_eval_results(artificial_evalset_df):
    cols = [c for c in artificial_evalset_df.columns.tolist() if c.startswith("preds-")]

    for col in cols:
        model_name = col[6:]
        print("#" * 60)
        print("Model:", model_name)
        print("#" * 60)
        print()
        
        labels = artificial_evalset_df["is_same_side"].values
        preds = artificial_evalset_df[col].values
        print("All (uninverted NEG labels):")
        compute_metrics(confusion_matrix(labels, preds))
        print()
        
        labels, preds = zip(*[(l, p) if l == 1 else (1, abs(p - 1)) for l, p in zip(labels, preds)])
        conf_mat = confusion_matrix(labels, preds)
        print("All:")            
        compute_metrics(conf_mat)
        print()

        for crit, crit_df in artificial_evalset_df.groupby("type"):
            crit_df = crit_df[["is_same_side", col]].astype({"is_same_side": "int32"})
            labels = crit_df["is_same_side"].values
            preds = crit_df[col].values

            if "NEG" in crit:
                # invert values for conf_mat
                labels = [abs(v - 1) for v in labels]
                preds = [abs(v - 1) for v in preds]

            conf_mat = confusion_matrix(labels, preds)
            print("Criterion:", crit)            
            compute_metrics(conf_mat)
            print()

        # all negs
        neg_cols = [c for c in artificial_evalset_df["type"].unique().tolist() if "NEG" in c]
        all_neg_df = artificial_evalset_df.loc[[
            any(v) for v in zip(*[
                artificial_evalset_df["type"] == c for c in neg_cols
            ])
        ]]
        labels = [abs(v - 1) for v in all_neg_df["is_same_side"].values]
        preds = [abs(v - 1) for v in all_neg_df[col].values]
        conf_mat = confusion_matrix(labels, preds)
        print("All negs:", neg_cols)            
        compute_metrics(conf_mat)
        print()
        
        # all pos
        pos_cols = [c for c in artificial_evalset_df["type"].unique().tolist() if "NEG" not in c]
        all_pos_df = artificial_evalset_df.loc[[
            any(v) for v in zip(*[
                artificial_evalset_df["type"] == c for c in pos_cols
            ])
        ]]
        labels = [v for v in all_pos_df["is_same_side"].values]
        preds = [v for v in all_pos_df[col].values]
        conf_mat = confusion_matrix(labels, preds)
        print("All pos:", pos_cols)            
        compute_metrics(conf_mat)
        print()

        print("\n")

In [47]:
dump_art_eval_results(artificial_evalset_df)

############################################################
Model: cross_traindev_epi512_BCE_0.1
############################################################

All (uninverted NEG labels):
  accuracy: 0.497
 precision: 0.447
    recall: 0.733
  f1-score: 0.556

All:
  accuracy: 0.497
 precision: 1.000
    recall: 0.497
  f1-score: 0.664

Criterion: CIT
  accuracy: 0.760
 precision: 1.000
    recall: 0.760
  f1-score: 0.864

Criterion: CIT-NEG
  accuracy: 0.320
 precision: 1.000
    recall: 0.320
  f1-score: 0.485

Criterion: CON
  accuracy: 0.920
 precision: 1.000
    recall: 0.920
  f1-score: 0.958

Criterion: CON-NEG
  accuracy: 0.320
 precision: 1.000
    recall: 0.320
  f1-score: 0.485

Criterion: DIFF
  accuracy: 0.520
 precision: 1.000
    recall: 0.520
  f1-score: 0.684

Criterion: DIFF-NEG
  accuracy: 0.440
 precision: 1.000
    recall: 0.440
  f1-score: 0.611

Criterion: NEG
  accuracy: 0.200
 precision: 1.000
    recall: 0.200
  f1-score: 0.333

All negs: ['NEG', 'CON-NEG', '